In [56]:
import pandas as pd

In [57]:
hotels = pd.read_csv('data/hotels_encoded_v2.csv', index_col=0)
hotels.head(3)

,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,population_ratio,distance_to_capital,popular_tags_count,hotel_name_0,hotel_name_1,...,country_United Kingdom,country_France,country_Austria,country_Mexico,neg,pos,compound,sample,reviewer_score,is_capital
0,8.1,52,16670,62,1,58887359,484.0,4,0,0,...,0,0,0,0,0.049,0.334,0.9924,0,0.0,0
1,8.6,10,5018,0,7,17797060,4.0,4,0,0,...,0,0,0,0,0.196,0.000,-0.2960,0,0.0,1
2,8.3,3,351,5,15,47615034,506.0,5,0,0,...,0,0,0,0,0.000,0.655,0.6908,0,0.0,0


In [58]:
RANDOM_SEED = 42

In [59]:
train_data = hotels[hotels['sample'] == 1].drop(columns='sample')
test_data = hotels[hotels['sample'] == 0].drop(columns='sample')

X = train_data.drop(columns='reviewer_score')
y = train_data['reviewer_score']

In [60]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
# X = hotels.drop(['reviewer_score'], axis=1)  
# y = hotels['reviewer_score'] 

In [61]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [62]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [63]:
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((128935, 31), (386803, 31), (386803, 30), (309442, 30), (77361, 30))

In [64]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
      
# Обучаем модель на тестовом наборе данных  
model.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = model.predict(X_test)  


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   36.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.4s finished


In [65]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.12773889708823769


Небольшой бонус:


In [66]:
# # # убираем признаки которые еще не успели обработать, 
# # # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis=1, inplace=True)

# # # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)